In [2]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, TensorDataset


from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [3]:
def retrieve_train_data():
    train_param = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/param/train.json")
    train_return = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/return/train.json")
    train_summary = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/summary/train.json")
    train_df = pd.concat([train_summary,train_param, train_return], axis=0)
    train_df = train_df.reset_index(drop=True)
    return train_df
def retrieve_test_data():
    valid_param = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/param/test.json")
    valid_return = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/return/test.json")
    valid_summary = pd.read_json("D:/BERT_learing/code_comment_inconsistency_detection/data/summary/test.json")
    valid_df = pd.concat([valid_summary,valid_param, valid_return ], axis=0)
    valid_df = valid_df.reset_index(drop=True)
    return valid_df

In [4]:
train_df = retrieve_train_data()
test_df = retrieve_test_data()

In [5]:
# 应该去掉多余空格和注释中的标点
def format_data(df):
    old_code_raw = df['new_code_raw']
    old_code_raw = old_code_raw.values
    old_code_raw = [str(ele) for ele in old_code_raw]
       
    multi_line_old_code = []
    for i in range(len(old_code_raw)):
        multi_line_test = old_code_raw[i].replace('\n', ' ')   # 去掉\n
        multi_line_test = ' '.join(multi_line_test.split())    # 把多余空格变成一个空格
        multi_line_old_code.append(multi_line_test) 
     
    old_comment_raw = df['old_comment_raw']
    old_comment_raw = old_comment_raw.values
    old_comment_raw = [str(ele) for ele in old_comment_raw]
    multi_line_old_comment = []
    for i in range(len(old_comment_raw)):
        multi_line_test = ' '.join(old_comment_raw[i].split())    # 把多余空格变成一个空格
        multi_line_old_comment.append(multi_line_test)  
    
    df['new_code_raw'] = multi_line_old_code
    df['old_comment_raw'] = multi_line_old_comment
    
    return df

In [6]:
train_df_clean = format_data(train_df)
test_df_clean = format_data(test_df)

In [7]:
train_df_clean['old_comment_raw'][0]

'Parses the given JSON and returns ether a JSONObject or a JSONArry'

In [8]:
train_df_clean['old_comment_raw'][210]

'Retrieve the Set of Conditions for this rule.'

In [9]:
list_find_index = [
    'Returns a OrientBaseGraph implementation from the current database in thread local.',
    'Creates a exception tag based on the Class # getSimpleName() simple name of the class of the givenexception.',
    'Clones a KeyValue.',
    'Lock a block with lock id.' ,
    'Returns the LikeBinding for the given position.',
    'Gets the context menu on the given control , matching the text.',
    'Returns an infinitely long Stream of int values starting from from.',
    'Returns a Completable instance that subscribes to all sources at once and completes only when all source Completables complete or one of them emits an error . ',
    'Shortcut method to get the config from com . sonyericsson . hudson . plugins . gerit . tigger . certServer.',  
    'Creates a net . bytebuddy . implementation . DefaultMethodCall implementation which searches the given list of interface types for a suitable default method in their order .'
]

In [10]:
str_2 =  'Returns a OrientBaseGraph implementation from the current database in thread local.' 

In [11]:
for i in range(len(train_df_clean)):
    temp1 = train_df_clean['old_comment_raw'][i]
    for j in list_find_index:
        temp2 = j.replace(" ", "")
        temp1 = temp1.replace(" ", "")
        if temp2 == temp1:
            print(train_df_clean['old_comment_raw'][i])
            print(i)

Creates a net.bytebuddy.implementation.DefaultMethodCall implementation which searches the given list of interface types for a suitable default method in their order.
363
Returns a OrientBaseGraph implementation from the current database in thread local.
610
Creates a exception tag based on the Class#getSimpleName() simple name of the class of the given exception.
716
Clones a KeyValue.
2074
Lock a block with lock id.
2535
Returns the LikeBinding for the given position.
2875
Returns an infinitely long Stream of int values starting from from.
2955
Gets the context menu on the given control, matching the text.
3539
Creates a net.bytebuddy.implementation.DefaultMethodCall implementation which searches the given list of interface types for a suitable default method in their order.
3746
Returns a Completable instance that subscribes to all sources at once and completes only when all source Completables complete or one of them emits an error.
4636


In [12]:
test_df_clean['old_comment_raw'][65]

'Returns all currently displayed sentences in plain text form.'

In [13]:
test_df_clean['new_comment_raw'][65]

'Returns all currently displayed sentences in string buffer, plain text form'

In [14]:
train_df_clean['old_comment_raw'][363]

'Creates a net.bytebuddy.implementation.DefaultMethodCall implementation which searches the given list of interface types for a suitable default method in their order.'

In [15]:
train_df_clean['new_comment_raw'][363]

'Creates a  net.bytebuddy.implementation.DefaultMethodCall implementation which searches the given list of interface types for a suitable default method in their order.'

In [16]:
train_df_clean['new_code_raw'][363]

'public static Implementation prioritize(Iterable<? extends Class<?>> prioritizedInterfaces) { List<Class<?>> list = new ArrayList<Class<?>>(); for (Class<?> prioritizedInterface : prioritizedInterfaces) { list.add(prioritizedInterface); } return prioritize(new TypeList.ForLoadedTypes(list)); }'

In [17]:
train_df_clean.loc[363]

id                                                 raphw_byte_buddy-1042-FirstSentence-0
label                                                                                  0
comment_type                                                                     Summary
old_comment_raw                        Creates a net.bytebuddy.implementation.Default...
old_comment_subtokens                  [creates, a, net, ., bytebuddy, ., implementat...
new_comment_raw                        Creates a  net.bytebuddy.implementation.Defaul...
new_comment_subtokens                  [creates, a, net, ., bytebuddy, ., implementat...
span_minimal_diff_comment_subtokens                                                   []
old_code_raw                               public static Implementation prioritize(It...
old_code_subtokens                     [public, static, implementation, prioritize, (...
new_code_raw                           public static Implementation prioritize(Iterab...
new_code_subtokens   

In [18]:
train_df_clean_test = train_df_clean[360:364]
train_df_clean_test = train_df_clean_test.reset_index(drop=True)

train_df_clean_test.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens
0,mybatis_spring-79-FirstSentence-0,1,Summary,Gets an SqlSession from Spring Transaction Man...,"[gets, an, sql, session, from, spring, transac...",Creates a new MyBatis SqlSession from the Sq...,"[creates, a, new, my, batis, sql, session, fro...","[<REPLACE_OLD>, gets, an, <REPLACE_NEW>, creat...",public static SqlSession getSqlSession(SqlSe...,"[public, static, sql, session, get, sql, sessi...",public static SqlSession getSqlSession(SqlSess...,"[public, static, sql, session, get, sql, sessi...","[<KEEP>, public, static, sql, session, get, sq...","[<KEEP>, public, <KEEP>, static, <KEEP>, sql, ..."
1,googlemaps_android_maps_utils-150-FirstSentence-0,0,Summary,Get the marker from a Cluster,"[get, the, marker, from, a, cluster]",Get the marker from a Cluster,"[get, the, marker, from, a, cluster]",[],public Marker getMarker(Cluster<T> cluster...,"[public, marker, get, marker, (, cluster, <, t...",public Marker getMarker(Cluster<T> cluster) { ...,"[public, marker, get, marker, (, cluster, <, t...","[<KEEP>, public, marker, get, marker, (, clust...","[<KEEP>, public, <KEEP>, marker, <KEEP>, get, ..."
2,apache_tomcat-3092-FirstSentence-0,1,Summary,"Read one constant from the given file, the typ...","[read, one, constant, from, the, given, file, ...","Read one constant from the given input, the ty...","[read, one, constant, from, the, given, input,...","[<REPLACE_OLD>, file, <REPLACE_NEW>, input, <R...",static Constant readConstant(DataInput fil...,"[static, constant, read, constant, (, data, in...",static Constant readConstant(DataInput input) ...,"[static, constant, read, constant, (, data, in...","[<KEEP>, static, constant, read, constant, (, ...","[<KEEP>, static, <KEEP>, constant, <KEEP>, rea..."
3,raphw_byte_buddy-1042-FirstSentence-0,0,Summary,Creates a net.bytebuddy.implementation.Default...,"[creates, a, net, ., bytebuddy, ., implementat...",Creates a net.bytebuddy.implementation.Defaul...,"[creates, a, net, ., bytebuddy, ., implementat...",[],public static Implementation prioritize(It...,"[public, static, implementation, prioritize, (...",public static Implementation prioritize(Iterab...,"[public, static, implementation, prioritize, (...","[<KEEP>, public, static, implementation, prior...","[<KEEP>, public, <KEEP>, static, <KEEP>, imple..."


In [19]:
import javalang
# 用这个，这个对了
def get_blocks_improved_BFS_code(root):
    queue = []
    nodes = []
    queue.append(root)
    level = 0
    while queue:
        level = level + 1
        for i in range(len(queue)):
            node = queue.pop(0)
            nodes.append(node)
            if hasattr(node, 'children'):
                for child in node.children:
                    queue.append(child)
            else:
                try:
                    len_node = len(node)
                    for i in range(len_node):
                        if hasattr(node[i], 'children'):
                            for child in node[i].children:
                                 queue.append(child)
                except Exception as e:
                    continue
    for node in nodes:
        if isinstance(node, javalang.tree.Node):
            index = nodes.index(node)
            nodes[index] = type(node).__name__
        if isinstance(node, list):
            for no in node:
                if isinstance(no, javalang.tree.Node):
                    index = node.index(no)
                    node[index] = type(no).__name__

            
    return level,nodes
# 使用javalang解析数据，将AST的层数,和ASTnodes(ast层次遍历结果)  变为dataframe中的一列，返回新的dataframe

def get_BFS_df(df):

    error_index = []
    asts = []
    levels = []
    nodes = []
    for i in range(len(df)):
        tokens_t = javalang.tokenizer.tokenize(df.loc[i]['new_code_raw'])
        try:
            parser = javalang.parse.Parser(tokens_t)
        except Exception as e:
            error_index.append(i)
            continue
        asts.append(parser)  

    for i in range(len(asts)):
        try:
            # 尝试解析Java代码
            test = asts[i].parse_member_declaration()
            depth,node = get_blocks_improved_BFS_code(test)
            levels.append(depth) 
            nodes.append(node)
        except javalang.parser.JavaSyntaxError:
            # 如果出现JavaSyntaxError，打印错误并跳过
            error_index.append(i)
            print(f"解析错误索引为ast[{i}]，跳过有问题的代码")
            continue 
            
    for i in error_index:
        df.drop(i, inplace=True)
    df = df.reset_index(drop=True)
    
    df['AST_level'] = levels
    df['AST_BFSnodes'] = nodes
    return df
test_df_ast = get_BFS_df(train_df_clean_test)
test_df_ast.head()

解析错误索引为ast[2]，跳过有问题的代码


,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes
0,mybatis_spring-79-FirstSentence-0,1,Summary,Gets an SqlSession from Spring Transaction Man...,"[gets, an, sql, session, from, spring, transac...",Creates a new MyBatis SqlSession from the Sq...,"[creates, a, new, my, batis, sql, session, fro...","[<REPLACE_OLD>, gets, an, <REPLACE_NEW>, creat...",public static SqlSession getSqlSession(SqlSe...,"[public, static, sql, session, get, sql, sessi...",public static SqlSession getSqlSession(SqlSess...,"[public, static, sql, session, get, sql, sessi...","[<KEEP>, public, static, sql, session, get, sq...","[<KEEP>, public, <KEEP>, static, <KEEP>, sql, ...",6,"[MethodDeclaration, None, {static, public}, []..."
1,googlemaps_android_maps_utils-150-FirstSentence-0,0,Summary,Get the marker from a Cluster,"[get, the, marker, from, a, cluster]",Get the marker from a Cluster,"[get, the, marker, from, a, cluster]",[],public Marker getMarker(Cluster<T> cluster...,"[public, marker, get, marker, (, cluster, <, t...",public Marker getMarker(Cluster<T> cluster) { ...,"[public, marker, get, marker, (, cluster, <, t...","[<KEEP>, public, marker, get, marker, (, clust...","[<KEEP>, public, <KEEP>, marker, <KEEP>, get, ...",6,"[MethodDeclaration, None, {public}, [], None, ..."
2,raphw_byte_buddy-1042-FirstSentence-0,0,Summary,Creates a net.bytebuddy.implementation.Default...,"[creates, a, net, ., bytebuddy, ., implementat...",Creates a net.bytebuddy.implementation.Defaul...,"[creates, a, net, ., bytebuddy, ., implementat...",[],public static Implementation prioritize(It...,"[public, static, implementation, prioritize, (...",public static Implementation prioritize(Iterab...,"[public, static, implementation, prioritize, (...","[<KEEP>, public, static, implementation, prior...","[<KEEP>, public, <KEEP>, static, <KEEP>, imple...",9,"[MethodDeclaration, None, {static, public}, []..."


In [22]:
train_df_clean_test = train_df_clean[716:717]
train_df_clean_test = train_df_clean_test.reset_index(drop=True)

test_df_ast = get_BFS_df(train_df_clean_test)
test_df_ast.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes
0,spring_projects_spring_boot-882-FirstSentence-0,0,Summary,Creates a exception tag based on the Class#get...,"[creates, a, exception, tag, based, on, the, c...",Creates a exception tag based on the Class#g...,"[creates, a, exception, tag, based, on, the, c...",[],\tpublic static Tag exception(Throwable except...,"[public, static, tag, exception, (, throwable,...",public static Tag exception(Throwable exceptio...,"[public, static, tag, exception, (, throwable,...","[<KEEP>, public, static, tag, exception, (, th...","[<KEEP>, public, <KEEP>, static, <KEEP>, tag, ...",9,"[MethodDeclaration, None, {static, public}, []..."


In [23]:
test_df_ast['AST_BFSnodes'][0]

['MethodDeclaration',
 None,
 {'public', 'static'},
 [],
 None,
 'ReferenceType',
 'exception',
 ['FormalParameter'],
 None,
 ['IfStatement', 'ReturnStatement'],
 'Tag',
 [],
 None,
 None,
 set(),
 [],
 'ReferenceType',
 'exception',
 False,
 None,
 'BinaryOperation',
 'BlockStatement',
 None,
 None,
 'MemberReference',
 'Throwable',
 [],
 None,
 None,
 '!=',
 'MemberReference',
 'Literal',
 None,
 ['LocalVariableDeclaration', 'ReturnStatement'],
 [],
 [],
 '',
 [],
 'EXCEPTION_NONE',
 [],
 [],
 '',
 [],
 'exception',
 [],
 [],
 None,
 [],
 'null',
 set(),
 [],
 'ReferenceType',
 ['VariableDeclarator'],
 None,
 'MethodInvocation',
 'String',
 [],
 None,
 None,
 'simpleName',
 [],
 'MethodInvocation',
 [],
 [],
 'Tag',
 [],
 None,
 ['Literal', 'TernaryExpression'],
 'of',
 [],
 [],
 'exception',
 ['MethodInvocation'],
 None,
 [],
 'getClass',
 [],
 [],
 None,
 [],
 '"exception"',
 'MethodInvocation',
 'MethodInvocation',
 'MemberReference',
 None,
 None,
 None,
 None,
 None,
 [],
 'getS

In [220]:
train_df_clean_test = train_df_clean[363:364]
train_df_clean_test = train_df_clean_test.reset_index(drop=True)

train_df_clean_test.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens
0,raphw_byte_buddy-1042-FirstSentence-0,0,Summary,Creates a net.bytebuddy.implementation.Default...,"[creates, a, net, ., bytebuddy, ., implementat...",Creates a net.bytebuddy.implementation.Defaul...,"[creates, a, net, ., bytebuddy, ., implementat...",[],public static Implementation prioritize(It...,"[public, static, implementation, prioritize, (...",public static Implementation prioritize(Iterab...,"[public, static, implementation, prioritize, (...","[<KEEP>, public, static, implementation, prior...","[<KEEP>, public, <KEEP>, static, <KEEP>, imple..."


In [222]:
train_df_1 = train_df_clean[610:611]
train_df_2 = train_df_clean[716:717]
test_df_3 = test_df_clean[65:66]
train_df_4 = train_df_clean[363:364]

test_df = pd.concat([train_df_1,train_df_2, test_df_3,train_df_4], axis=0)
test_df = test_df.reset_index(drop=True)

test_df.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens
0,orientechnologies_orientdb-1949-FirstSentence-0,1,Summary,Returns a OrientBaseGraph implementation from ...,"[returns, a, orient, base, graph, implementati...",Returns a Transactional OrientGraph implementa...,"[returns, a, transactional, orient, graph, imp...","[<INSERT_OLD_KEEP_BEFORE>, a, <INSERT_NEW_KEEP...",public static OrientBaseGraph getGraph() {\n...,"[public, static, orient, base, graph, get, gra...",public static OrientBaseGraph getGraph() { ODa...,"[public, static, orient, base, graph, get, gra...","[<KEEP>, public, static, orient, base, graph, ...","[<KEEP>, public, <KEEP>, static, <KEEP>, orien..."
1,spring_projects_spring_boot-882-FirstSentence-0,0,Summary,Creates a exception tag based on the Class#get...,"[creates, a, exception, tag, based, on, the, c...",Creates a exception tag based on the Class#g...,"[creates, a, exception, tag, based, on, the, c...",[],\tpublic static Tag exception(Throwable except...,"[public, static, tag, exception, (, throwable,...",public static Tag exception(Throwable exceptio...,"[public, static, tag, exception, (, throwable,...","[<KEEP>, public, static, tag, exception, (, th...","[<KEEP>, public, <KEEP>, static, <KEEP>, tag, ..."
2,stanfordnlp_CoreNLP-191-Associations-FirstSent...,1,Summary,Returns all currently displayed sentences in p...,"[returns, all, currently, displayed, sentences...",Returns all currently displayed sentences in s...,"[returns, all, currently, displayed, sentences...","[<INSERT_OLD_KEEP_BEFORE>, in, <INSERT_NEW_KEE...",public String getMatchedSentences() {\n S...,"[public, string, get, matched, sentences, (, )...",public StringBuffer getMatchedSentences() { St...,"[public, string, buffer, get, matched, sentenc...","[<KEEP>, public, string, <KEEP_END>, <INSERT>,...","[<KEEP>, public, <KEEP>, string, <INSERT>, buf..."
3,raphw_byte_buddy-1042-FirstSentence-0,0,Summary,Creates a net.bytebuddy.implementation.Default...,"[creates, a, net, ., bytebuddy, ., implementat...",Creates a net.bytebuddy.implementation.Defaul...,"[creates, a, net, ., bytebuddy, ., implementat...",[],public static Implementation prioritize(It...,"[public, static, implementation, prioritize, (...",public static Implementation prioritize(Iterab...,"[public, static, implementation, prioritize, (...","[<KEEP>, public, static, implementation, prior...","[<KEEP>, public, <KEEP>, static, <KEEP>, imple..."


In [224]:
test_df_ast = get_BFS_df(test_df)
test_df_ast.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens,AST_level,AST_BFSnodes
0,orientechnologies_orientdb-1949-FirstSentence-0,1,Summary,Returns a OrientBaseGraph implementation from ...,"[returns, a, orient, base, graph, implementati...",Returns a Transactional OrientGraph implementa...,"[returns, a, transactional, orient, graph, imp...","[<INSERT_OLD_KEEP_BEFORE>, a, <INSERT_NEW_KEEP...",public static OrientBaseGraph getGraph() {\n...,"[public, static, orient, base, graph, get, gra...",public static OrientBaseGraph getGraph() { ODa...,"[public, static, orient, base, graph, get, gra...","[<KEEP>, public, static, orient, base, graph, ...","[<KEEP>, public, <KEEP>, static, <KEEP>, orien...",8,"[MethodDeclaration, None, {static, public}, []..."
1,spring_projects_spring_boot-882-FirstSentence-0,0,Summary,Creates a exception tag based on the Class#get...,"[creates, a, exception, tag, based, on, the, c...",Creates a exception tag based on the Class#g...,"[creates, a, exception, tag, based, on, the, c...",[],\tpublic static Tag exception(Throwable except...,"[public, static, tag, exception, (, throwable,...",public static Tag exception(Throwable exceptio...,"[public, static, tag, exception, (, throwable,...","[<KEEP>, public, static, tag, exception, (, th...","[<KEEP>, public, <KEEP>, static, <KEEP>, tag, ...",9,"[MethodDeclaration, None, {static, public}, []..."
2,stanfordnlp_CoreNLP-191-Associations-FirstSent...,1,Summary,Returns all currently displayed sentences in p...,"[returns, all, currently, displayed, sentences...",Returns all currently displayed sentences in s...,"[returns, all, currently, displayed, sentences...","[<INSERT_OLD_KEEP_BEFORE>, in, <INSERT_NEW_KEE...",public String getMatchedSentences() {\n S...,"[public, string, get, matched, sentences, (, )...",public StringBuffer getMatchedSentences() { St...,"[public, string, buffer, get, matched, sentenc...","[<KEEP>, public, string, <KEEP_END>, <INSERT>,...","[<KEEP>, public, <KEEP>, string, <INSERT>, buf...",10,"[MethodDeclaration, None, {public}, [], None, ..."
3,raphw_byte_buddy-1042-FirstSentence-0,0,Summary,Creates a net.bytebuddy.implementation.Default...,"[creates, a, net, ., bytebuddy, ., implementat...",Creates a net.bytebuddy.implementation.Defaul...,"[creates, a, net, ., bytebuddy, ., implementat...",[],public static Implementation prioritize(It...,"[public, static, implementation, prioritize, (...",public static Implementation prioritize(Iterab...,"[public, static, implementation, prioritize, (...","[<KEEP>, public, static, implementation, prior...","[<KEEP>, public, <KEEP>, static, <KEEP>, imple...",9,"[MethodDeclaration, None, {static, public}, []..."


In [208]:
list_find_index = [
    ' public static Implementation prioritize (Iterable > prioritizedInterfaces ) { List < Class > list = new ArrayList < Class >(); for ( Class prioritizedInterface:prioritizedInterfaces ) { list.add( prioritizedInterface ); } return prioritize ( new TypeList. ForLoadedTypes (list));} ',
   ]
for i in range(len(train_df_clean)):
    temp1 = train_df_clean['new_code_raw'][i]
    for j in list_find_index:
        temp2 = j.replace(" ", "")
        temp1 = temp1.replace(" ", "")
        if temp2 == temp1:
            print(train_df_clean['new_code_raw'][i])
            print(i)